<p style="text-align:center"><font size="10" >Kawase Seminar 06-2</font></p>

全2回にわけて実際の論文の手法を実践し、スクレイピングとテキストマイングの利用を体験していきます。

第6回でも、1.実際に現状を把握する, 2.解くための筋道を立てる, 3.Densuke(田村)が実装したコードを読んで2の筋道が実際に正しかったのか確認する, といったことを逐次行っていきます。

今回以降は、時間の関係上実装はそこまで行えませんがかなりの学びになるので、皆さんも各自でトライして頂けると良いと思われます💪

### **やること**
-  テキストマイニング分野における実装を大まかに考える
-  大まかな実装を細かい単位に分解し具体的なロジックを組み立てる
-  **ロジックをもとに実装する ->イマココ**


今回のコードを読む目的としては、どのように田村が実装したのかを田村の思考を理解し、

実際に自分で書く際の糧になってもらいことにあります。


---
## **田村のTODOメモ**

1. 第１段階、第２段階の各段階で収集したメッセージデータの中から1000個のメッセージをランダムに選択
    1.  ランダムに企業コードを選択するプログラムを実行し、選択された企業名からcsvデータを読み込む
    2. 読み込まれたcsvデータのメッセージの項目から適当に1つ選択する
    3. 1, 2を1000回繰り返す
　
2. 1000個のメッセージを手動で強気、弱気、中立に分類

3. ２で、分類されたメッセージの中から、「売り」「買い」「高」など、強気・弱気の感情に大きく関係するキーワードを選択。

4. macabなどの形態素解析ライブラリでメッセージの単語を分割する(その際に情報やノイズの多い情報を除外するために167の単語を辞書に追加)

5. 企業数分のメッセージ（形態素解析で分けられた単語群）配列をキーワード頻度行列に変換

6. SVM利用し、3つのセンチメントクラスに分類

___


## 実装が可能な部分を踏まえたTODO

今回、時間内や調査の関係上実装が難しいところがあるため、内容を変更してTODOを再作成しました。

1. 企業のcsvデータから、macabなどの形態素解析ライブラリでメッセージの単語を分割
2. メッセージをplus(1>=N),  minus(N<= -1 ), normal(±0) に分類
3.  plus, minus, normalに分けたメッセージにある単語を多い順にソートする

✴︎今回はSVMで学習させて分類するのではなく、すでに学習されたモデル(日本語評価極性辞書)を利用し分類
https://www.cl.ecei.tohoku.ac.jp/Open_Resources-Japanese_Sentiment_Polarity_Dictionary.html

---
# 1. 企業のcsvデータから、macabなどの形態素解析ライブラリでメッセージの単語を分割

> 形態素解析とは、自然言語処理（NLP）の一部で、自然言語で書かれた文を言語上で意味を持つ最小単位(＝形態素)に分け、それぞれの品詞や変化などを判別することです。
例えば　「庭には二羽ニワトリがいる」という文章を庭（名詞）/に（助詞）/は（助詞）/二（数詞）/羽（助数詞）/ニワトリ（名詞）/が（助詞）/いる（動詞）のように形態素に分解し、意味を割り出します。
https://ledge.ai/morpho_analysis_japan/


今回は形態素解析の方法の一つとしてオープンソースのmecabを利用
https://qiita.com/taroc/items/b9afd914432da08dafc8

In [3]:
!pip install mecab

  Using cached mecab-0.996.3.tar.gz (62 kB)
    ERROR: Command errored out with exit status 1:
     command: /opt/conda/bin/python3.8 -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-ni9z0ssd/mecab_6d5a05e438ea4c86b364223e7a4c16cf/setup.py'"'"'; __file__='"'"'/tmp/pip-install-ni9z0ssd/mecab_6d5a05e438ea4c86b364223e7a4c16cf/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /tmp/pip-pip-egg-info-8w3ru3jr
         cwd: /tmp/pip-install-ni9z0ssd/mecab_6d5a05e438ea4c86b364223e7a4c16cf/
    Complete output (10 lines):
    /bin/sh: 1: mecab-config: not found
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "/tmp/pip-install-ni9z0ssd/mecab_6d5a05e438ea4c86b364223e7a4c16cf/setup.py", line 53, in <module>
        include_dirs=cmd2("mecab-config --inc-dir"),
      File "/tmp/pip-install

In [ ]:
import MeCab

tagger = MeCab.Tagger("-Owakati") 
text = "この木なんの木気になる木"
print(tagger.parse(text).split())

**形態素解析をするメッセージデータのアップロード**

今回は、前回取得したメッセージデータを採用。
https://drive.google.com/file/d/1r4DWXDwqluxhxnnaFS9Pdoh5lNND0Ua-/view?usp=sharing

In [ ]:
df = pd.read_csv("東和ハイシステム(株).csv", index_col=0)
df

In [ ]:
func = lambda x: tagger.parse(x).split()
df["Text"] = df["Text"].map(func)
df

# 2. メッセージをplus(1>=N), minus(N<= -1 ), normal(±0) に分類

今回は東北大学の乾・鈴木研究室のページで公開されている日本語評価極性辞書を使った

Sentiment Analysis (いわゆるネガポジ判定) ライブラリ oseti を採用。
https://qiita.com/yukinoi/items/46aa016d83bb0e64f598

In [ ]:
!pip install oseti

In [ ]:
import oseti

analyzer = oseti.Analyzer()

analyzer.analyze('天国で待ってる。')

analyzer.analyze('遅刻したけど楽しかったし嬉しかった。すごく充実した！')

In [ ]:
func = lambda x: analyzer.analyze(x)[0]
df["Text_emotion_val"] = df["Text"].map(func)
df

# 3. plus, minus, normalに分けたメッセージにある単語を多い順にソートする

In [ ]:
plus = df[df['Text_emotion_val'] > 0]
plus = plus.copy()
normal = df[df['Text_emotion_val'] == 0]
normal = normal.copy()
minus = df[df['Text_emotion_val'] < 0]
minus = minus.copy()

In [ ]:
plus 

In [ ]:
normal

In [ ]:
minus

In [ ]:
import MeCab
tagger = MeCab.Tagger("-Ochasen") 

def convert_text_nouns(text) :
    nouns = [line.split()[0] for line in tagger.parse(text).splitlines() if "名詞" in line.split()[-1]]
    return nouns 

for data in [plus, normal, minus]:
    data["splited_data"] = data["Text"].map(convert_text_nouns)
    
    word_counts = {} 
    for values in data["splited_data"] :
        for val in values :
            if  val  in word_counts :
                word_counts[val] += 1 
            else:
                word_counts[val] = 1 
            
    
    score_sorted = sorted(word_counts.items(), key=lambda x:x[1], reverse=True)
    
    print(score_sorted)
    print("-" * 100)

### Reference
- https://note.com/junmaeda/n/n39b8b286da13

- https://qiita.com/g-k/items/1b7c765fa6520297ca7c

# Closing remarks


[![Image from Gyazo](https://i.gyazo.com/301c46663a66c700eea1aa066e4a0e2b.png)](https://gyazo.com/301c46663a66c700eea1aa066e4a0e2b)

[![Image from Gyazo](https://i.gyazo.com/27cc5af78af3ef4d7a4244f493227c8d.png)](https://gyazo.com/27cc5af78af3ef4d7a4244f493227c8d)

[![Image from Gyazo](https://i.gyazo.com/64f2c5739bbd4f6ac04c0b6e93ce3ee2.png)](https://gyazo.com/64f2c5739bbd4f6ac04c0b6e93ce3ee2)

[![Image from Gyazo](https://i.gyazo.com/ee800b74e2f49d75013c18e0705ea149.png)](https://gyazo.com/ee800b74e2f49d75013c18e0705ea149)


<h4 style="text-align:center;"> みなさんも実際にプログラムを作成してみてください!!!! 良いデータサイエンスライフを!!!!💪 </h4>


[![Image from Gyazo](https://i.gyazo.com/d9b57ca7fe3bc438fa9a7c60ca027940.png)](https://gyazo.com/d9b57ca7fe3bc438fa9a7c60ca027940)

https://github.com/Densuke-fitness

---
---
---
---

## Additional implementations: Tamura Challenge

せっかくなので、手動でデータを分類することを見越して下記のTODOが実行可能な状態に持っていけるようにします。

今回はディスカッションしながらゴールに辿りこうと思います。


1. 第１段階、第２段階の各段階で収集したメッセージデータの中から1000個のメッセージをランダムに選択し、1つのdataframeに保存
    1.  ランダムに企業コードを選択するプログラムを実行し、選択された企業名からcsvデータを読み込む
    2. 読み込まれたcsvデータのメッセージの項目から適当に1つ選択する
    3. 1, 2を1000回繰り返す